In [2]:
import pandas as pd

### 데이터 베이스 설정

In [3]:
DB_URL = 'onad.cbjjamtlar2t.ap-northeast-2.rds.amazonaws.com'
DB_USER = 'onad'
DB_PASSWORD = 'rkdghktn12'
DB_DATABASE = 'onad'
DB_CHARSET = 'utf8mb4'
DB_LOGFLAG  = 'False'
DB_PORT = 3306

db_url = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    DB_USER, DB_PASSWORD,
    DB_URL, DB_PORT,
    DB_DATABASE, DB_CHARSET
)

In [4]:
def data_from_sql_twitchchat(streamer, target_date):
    query = """
        select * from twitch_chat where streamer_name = "%s" and broad_date ="%s"
    """% (streamer, target_date)

    return pd.read_sql_query(query, db_url)

## 채팅 데이터 로드

In [5]:
streamer = "yapyap30"
target_date = "2018-12-08"
chat_data = data_from_sql_twitchchat(streamer, target_date)

### Exploratory Data Analysis

In [6]:
chat_data.shape

(110398, 6)

In [7]:
chat_data.head()

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
0,8266174,yapyap30,2018-12-08,<뱃살몽땅 (yeppt7f)>,00:00:04,성태님 레바 ㄹㅇ 개쉽읍니다 홧팅
1,8266175,yapyap30,2018-12-08,<isolet77>,00:00:04,배소리안나니까 편안하네
2,8266176,yapyap30,2018-12-08,<문둥이 (mundung2)>,00:00:05,얍얍님 저도 58이었는데 저 대둡니다
3,8266177,yapyap30,2018-12-08,<kakle0428>,00:00:05,아니 키하고 어깨 좁으니까 58이라도 상대적으로 ㅆ머두지
4,8266178,yapyap30,2018-12-08,<리을비 (leech211)>,00:00:06,59쓰는애는 머리진짜 큰건데


In [8]:
print("채팅 로그의 수: %s" % (chat_data.shape[0]))
print("순수 시청자 수: %s" % (len(list(chat_data['chatterer'].unique()))))

채팅 로그의 수: 110398
순수 시청자 수: 6500


In [9]:
print("채팅로그의 시작과 끝 : %s ~ %s " % (sorted(chat_data['chat_time'])[0], sorted(chat_data['chat_time'])[-1]))

채팅로그의 시작과 끝 : 00:00:04 ~ 23:59:56 


### 시간데이터를 DatetimeIndex 로 변경

In [10]:
chat_data['time'] = pd.to_datetime(
    chat_data['broad_date'].apply(lambda x : ",".join(x.split("-"))) + "," + \
    chat_data['chat_time'].apply(lambda x : ",".join(x.split(":"))), 
                                  format="%Y,%m,%d,%H,%M,%S")


In [11]:
chat_data_reindex = chat_data.set_index('time')

### pd.Datetime index 생성

In [12]:
chat_data_reindex.head(2)

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
time,,,,,,
2018-12-08 00:00:04,8266174,yapyap30,2018-12-08,<뱃살몽땅 (yeppt7f)>,00:00:04,성태님 레바 ㄹㅇ 개쉽읍니다 홧팅
2018-12-08 00:00:04,8266175,yapyap30,2018-12-08,<isolet77>,00:00:04,배소리안나니까 편안하네


In [13]:
# YYYY-mm-dd HH:MM:SS 의 datetime 객체 인덱스 생성 (00:00:00 ~ 23:59:59 까지)
index=pd.date_range(target_date, periods=86400, freq="S")
# 인덱스만 가지는 데이터 프레임 생성 (chat_data와 병합하기 위함)
df = pd.DataFrame(index=index,)

In [14]:
# 필요없는 행 삭제
del chat_data_reindex['chat_id']
del chat_data_reindex['streamer_name']
del chat_data_reindex['broad_date']
del chat_data_reindex['chat_time']

### 한 문장에서 특정 단어의 수 세어 컬럼으로

In [15]:
chat_data_reindex['cnt_ㅋ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㅋ"))
chat_data_reindex['cnt_ㄵ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㄵ"))
chat_data_reindex['cnt_ㄴㅈ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㄴㅈ"))

In [16]:
pivot_df = chat_data_reindex.pivot_table(index=chat_data_reindex.index, aggfunc=len, values='chatterer')
pivot_df.columns = ['cnt_chat']
pivot_df["cnt_ㅋ"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㅋ')['cnt_ㅋ']
pivot_df["cnt_ㄵ"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㄵ')['cnt_ㄵ'] +\
                    chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㄵ')['cnt_ㄵ']

# 초당 채팅수가 6이상인 경우만
pivot_df[pivot_df['cnt_chat'] > pivot_df['cnt_chat'].mean()]

pivot_df['cnt_ㄵ'].unique()
pivot_df[pivot_df['cnt_ㄵ'] == 12]

,cnt_chat,cnt_ㅋ,cnt_ㄵ
time,,,
2018-12-08 04:35:56,10,0,12


In [17]:
chat_data_reindex[chat_data_reindex.index == "2018-12-08 04:35:56"]

,chatterer,chat_contents,cnt_ㅋ,cnt_ㄵ,cnt_ㄴㅈ
time,,,,,
2018-12-08 04:35:56,<갸꿀잼몰카 (qntksamtwoddl)>,ㄵ,0,1,0
2018-12-08 04:35:56,<모드림 (chingizk)>,ㄴㅈ,0,0,1
2018-12-08 04:35:56,<%Adonbbagadon>,ㄵ,0,1,0
2018-12-08 04:35:56,<조강현을받아들여라 (take_the_steelo)>,ㄵㄵ,0,2,0
2018-12-08 04:35:56,<+하이스푸 (smallbird_1)>,빯,0,0,0
2018-12-08 04:35:56,<케메 (tkddlzh)>,ㄵ,0,1,0
2018-12-08 04:35:56,<%우리성태_이름_불러줘 (lookgochu)>,개 노 잼,0,0,0
2018-12-08 04:35:56,<비틱 (rhdiddlxjf)>,아,0,0,0
2018-12-08 04:35:56,<%하랑미 (jys4180)>,angryyapOznojam,0,0,0


In [30]:
chat_data_reindex.head()

,chatterer,chat_contents,cnt_ㅋ,cnt_ㄵ,cnt_ㄴㅈ
time,,,,,
2018-12-08 00:00:04,<뱃살몽땅 (yeppt7f)>,성태님 레바 ㄹㅇ 개쉽읍니다 홧팅,0,0,0
2018-12-08 00:00:04,<isolet77>,배소리안나니까 편안하네,0,0,0
2018-12-08 00:00:05,<문둥이 (mundung2)>,얍얍님 저도 58이었는데 저 대둡니다,0,0,0
2018-12-08 00:00:05,<kakle0428>,아니 키하고 어깨 좁으니까 58이라도 상대적으로 ㅆ머두지,0,0,0
2018-12-08 00:00:06,<리을비 (leech211)>,59쓰는애는 머리진짜 큰건데,0,0,0


### 시작시간 불러와 시작시간으로부터 00:00:00 으로 시작하는 컬럼만들기

In [57]:
streamer_id = 66375105
query = "select * from twitch_stream where streamer_id = '%s'" % streamer_id
broad_date_df = pd.read_sql_query(query, db_url)

broad = pd.to_datetime(broad_date_df['broad_date'])
broad.apply(lambda x : x + pd.Timedelta("9hours"))  # 트위치 시간대는 UTC 기준이므로 한국시간으로 +9 하여 변경

0   2019-01-03 15:49:46
1   2019-01-04 10:59:13
2   2019-01-07 10:57:17
3   2019-01-07 17:53:12
Name: broad_date, dtype: datetime64[ns]